In [3]:
import requests
import pandas as pd
import sweetviz as sv
import os
import shutil

def request(url, params):
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error en la solicitud: {response.status_code}")
        return None

url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
params = {
    'format': 'geojson',
    'starttime': '2014-10-21',
    'endtime': '2015-10-21',
    'minmagnitude': 0,
    'limit': 20000,
}

response = request(url, params)

In [4]:
if response is not None:
    earthquakes = []
    for feature in response['features']:
        properties = feature['properties']
        geometry = feature['geometry']

        earthquake_info = {
            'place': properties['place'],
            'magnitude': properties['mag'],
            'time': pd.to_datetime(properties['time'], unit='ms'),
            'latitude': geometry['coordinates'][1],
            'longitude': geometry['coordinates'][0],
            'depth': geometry['coordinates'][2],
        }
        earthquakes.append(earthquake_info)

    df = pd.DataFrame(earthquakes)

    csv_path = 'results/earthquakes_last_10_years.csv'
    df.to_csv(csv_path, index=False)
    print(f"Datos guardados en '{csv_path}'.")

    excel_path = 'results/earthquakes_last_10_years.xlsx'
    df.to_excel(excel_path, index=False)
    print(f"Datos guardados en '{excel_path}'.")

    report = sv.analyze(df)  
    report.show_html('earthquake_data_report.html')  
    print("Análisis exploratorio de datos guardado en 'earthquake_data_report.html'.")


Datos guardados en 'earthquakes_last_10_years.csv'.


ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
from pycaret.regression import *
from sklearn.model_selection import train_test_split

df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
df['second'] = df['time'].dt.second

df = df.drop(columns=['place', 'time'])

X = df.drop(columns=['magnitude'])
y = df['magnitude']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

exp_reg = setup(data=train_data, target='magnitude', session_id=123,
                normalize=True,
                categorical_features=['year', 'month', 'day', 'hour', 'minute', 'second'])

best_model = compare_models()

final_model = create_model(best_model)

predictions = predict_model(final_model, data=test_data)

print(predictions[['magnitude']])

plot_model(final_model, plot='residuals')
plot_model(final_model, plot='error')

final_tuned_model = finalize_model(final_model)
save_model(final_tuned_model, 'best_earthquake_model')


df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

#Análisis Exploratorio de Datos con Sweetviz
# Comparar el dataset de entrenamiento con el de prueba, enfocándose en la columna 'magnitude'
my_report = sv.compare([df_train, "Training Data"], [df_test, "Test Data"], target_feat="magnitude")

#Mostrar y guardar el reporte como HTML
my_report.show_html('earthquake_comparison_report.html')
print("Análisis comparativo guardado en 'earthquake_comparison_report.html'.")


